# Goals and Tactics

Executing tactics in Pantograph is simple. To start a proof, call the
`Server.goal_start` function and supply an expression.

In [1]:
from pantograph import Server
from pantograph.expr import Site, TacticHave, TacticExpr, TacticMode

In [2]:
server = await Server.create()
state0 = await server.goal_start_async("forall (p q: Prop), Or p q -> Or q p")

This creates a *goal state*, which consists of some goals. In this
case since it is the beginning of a state, it has only one goal.

In [3]:
print(state0)


⊢ forall (p q: Prop), Or p q -> Or q p


To execute a tactic on a goal state, use `Server.goal_tactic`. This function
takes a state, a tactic, and an optional site (see below). Most Lean tactics are strings.

In [4]:
state1 = await server.goal_tactic_async(state0, "intro a")
print(state1)

a : Prop
⊢ ∀ (q : Prop), a ∨ q → q ∨ a


Executing a tactic produces a new goal state. If this goal state has no goals,
the proof is complete. You can recover the usual form of a goal with `str()`

In [5]:
print(state1.goals[0])

a : Prop
⊢ ∀ (q : Prop), a ∨ q → q ∨ a


## Error Handling and GC

When a tactic fails, it throws an exception:

In [6]:
try:
    state2 = await server.goal_tactic_async(state1, "assumption")
    print("Should not reach this")
except Exception as e:
    print(e)

["tactic 'assumption' failed\na : Prop\n⊢ ∀ (q : Prop), a ∨ q → q ∨ a"]


A state with no goals is considered solved

In [7]:
state0 = await server.goal_start_async("forall (p : Prop), p -> p")
state1 = await server.goal_tactic_async(state0, "intro")
state2 = await server.goal_tactic_async(state1, "intro h")
state3 = await server.goal_tactic_async(state2, "exact h")
state3

GoalState(state_id=5, goals=[], messages=[], _sentinel=[0, 1])

Execute `server.gc()` once in a while to delete unused goals.

In [8]:
await server.gc_async()

## Special Tactics

Lean has special provisions for some tactics. This includes `have`, `let`,
`calc`. To execute one of these tactics, create a `TacticHave`, `TacticLet`,
instance and feed it into `server.goal_tactic`.

Technically speaking `have` and `let` are not tactics in Lean, so their execution requires special attention.

In [9]:
state0 = await server.goal_start_async("1 + 1 = 2")
state1 = await server.goal_tactic_async(state0, TacticHave(branch="2 = 1 + 1", binder_name="h"))
print(state1)


⊢ 2 = 1 + 1
h : 2 = 1 + 1
⊢ 1 + 1 = 2


The `TacticExpr` "tactic" parses an expression and assigns it to the current
goal.  This leverages Lean's type unification system and is as expressive as
Lean expressions. Many proofs in Mathlib4 are written in a mixture of expression
and tactic forms.

In [10]:
state0 = await server.goal_start_async("forall (p : Prop), p -> p")
state1 = await server.goal_tactic_async(state0, "intro p")
state2 = await server.goal_tactic_async(state1, TacticExpr("fun h => h"))
print(state2)

## Sites

The optional `site` argument to `goal_tactic` controls the area of effect of a tactic. Site controls what the tactic sees when it asks Lean for the current goal. Most tactics only act on a single goal, but tactics acting on multiple goals are plausible as well (e.g. `simp_all`).

The `auto_resume` field defaults to the server option's `automaticMode` (which defaults to `True`). When this field is true, Pantograph will not deliberately hide other goals away from the tactic. This is the usual modus operandi of tactic proofs in Lean. When `auto_resume` is set to `False`, Pantograph will set other goals to dormant. This can be useful in limiting the area of effect of a tactic. However, dormanting a goal comes with the extra burden that it has to be activated ("resume") later, via `goal_resume`.

In [11]:
state = await server.goal_start_async("forall (p : Prop), p -> And p (Or p p)")
state = await server.goal_tactic_async(state, "intro p h")
state = await server.goal_tactic_async(state, "apply And.intro")
print(state)

left
p : Prop
h : p
⊢ p
right
p : Prop
h : p
⊢ p ∨ p


In the example below, we set `auto_resume` to `False`, and the sibling goal is dormanted.

In [12]:
state1 = await server.goal_tactic_async(state, "exact h", site=Site(goal_id=0, auto_resume=False))
print(state1)

In the example below, we preferentially operate on the second goal. Note that the first goal is still here.

In [13]:
state2 = await server.goal_tactic_async(state, "apply Or.inl", site=Site(goal_id=1))
print(state2)

right.h
p : Prop
h : p
⊢ p
left
p : Prop
h : p
⊢ p


## Tactic Modes

Pantograph has special provisions for handling `conv` and `calc` tactics. The commonality of these tactics is incremental feedback: The tactic can run half way and produce some goal. Pantograph supports this via tactic modes. Every goal carries around with it a `TacticMode`, and the user is free to switch between modes. By default, the mode is `TacticMode.TACTIC`.

In [14]:
state = await server.goal_start_async("∀ (a b: Nat), (b = 2) -> 1 + a + 1 = a + b")

state = await server.goal_tactic_async(state, "intro a b h")
state = await server.goal_tactic_async(state, TacticMode.CALC)
state = await server.goal_tactic_async(state, "1 + a + 1 = a + 1 + 1")
state

GoalState(state_id=19, goals=[Goal(id='_uniq.363', variables=[Variable(t='Nat', v=None, name='a'), Variable(t='Nat', v=None, name='b'), Variable(t='b = 2', v=None, name='h')], target='1 + a + 1 = a + 1 + 1', sibling_dep=None, name='calc', mode=<TacticMode.TACTIC: 1>), Goal(id='_uniq.382', variables=[Variable(t='Nat', v=None, name='a'), Variable(t='Nat', v=None, name='b'), Variable(t='b = 2', v=None, name='h')], target='a + 1 + 1 = a + b', sibling_dep=None, name=None, mode=<TacticMode.CALC: 3>)], messages=[], _sentinel=[2, 3, 6, 7, 4, 11, 12, 9, 10, 13, 16, 17, 18])